In [1]:
import findspark
findspark.init('/home/gani/spark-2.4.4-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession, functions as F
spark = SparkSession.builder.appName('flatenShip').getOrCreate()

In [26]:
df = spark.read.json('shipment.json', multiLine=True)

In [27]:
df.printSchema()

root
 |-- books: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- qty: long (nullable = true)
 |    |    |-- title: string (nullable = true)
 |-- customer: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- date: string (nullable = true)
 |-- shipmentId: long (nullable = true)
 |-- supplier: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- state: string (nullable = true)



In [19]:
df = df \
    .withColumn("supplier_name", F.col("supplier.name")) \
    .withColumn("supplier_city", F.col("supplier.city")) \
    .withColumn("supplier_state", F.col("supplier.state")) \
    .withColumn("supplier_country", F.col("supplier.country")) \
    .drop("supplier") \
    .withColumn("customer_name", F.col("customer.name")) \
    .withColumn("customer_city", F.col("customer.city")) \
    .withColumn("customer_state", F.col("customer.state")) \
    .withColumn("customer_country", F.col("customer.country")) \
    .drop("customer") \
    .withColumn('items',F.explode(F.col('books')))

In [8]:
df.head(1)

[Row(books=[Row(qty=2, title='Spark with Java'), Row(qty=25, title='Spark in Action, 2nd Edition'), Row(qty=1, title='Spark in Action, 1st Edition')], date='2019-10-05', shipmentId=458922, supplier_name='Manning Publications', supplier_city='Shelter Island', supplier_state='New York', supplier_country='USA', customer_name='Jean Georges Perrin', customer_city='Chapel Hill', customer_state='North Carolina', customer_country='USA', items=Row(qty=2, title='Spark with Java'))]

In [20]:
df = df.withColumn('qty', F.col('items.qty')) \
        .withColumn('title', F.col('items.title')) \
        .drop('items') \
        .drop('books')

In [21]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- shipmentId: long (nullable = true)
 |-- supplier_name: string (nullable = true)
 |-- supplier_city: string (nullable = true)
 |-- supplier_state: string (nullable = true)
 |-- supplier_country: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_country: string (nullable = true)
 |-- qty: long (nullable = true)
 |-- title: string (nullable = true)



In [22]:
df.createOrReplaceTempView('ship_detail')

In [23]:
sqlQuery = 'select count(*) as bookCount from ship_detail'

In [24]:
bookCountDF = spark.sql(sqlQuery)

In [25]:
bookCountDF.show()

+---------+
|bookCount|
+---------+
|        3|
+---------+

